In [1]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, sampler
from tqdm.notebook import tqdm_notebook
import copy
import transformers
from transformers import pipeline, AutoTokenizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
path = '/content/drive/MyDrive/Deep/HW4/Q3'
file = open(path+ '/ferdousi.txt', 'r')
File = file.read()
file.close()

In [5]:
data = File.split('\n')
data = data[2:-1]
N_total = len(data)
N = N_total//2

In [6]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [7]:
data_tokenized = tokenizer(data,  truncation= True, max_length= 13, padding = 'max_length')
data_tokenized = data_tokenized['input_ids']

In [9]:
len_beits = []
for i in range(2*N):
  len_beits.append(len(data_tokenized[i]))
max(len_beits)

13

In [10]:
Data = np.array(data_tokenized).reshape((-1,26))

In [12]:
class Ferdousi(Dataset):
  def __init__(self, Data, N = len(Data)-1):
    ind = np.random.choice(N, N, replace= False)
    self.x = Data[ind]
    self.y = Data[ind+1]
    self.num = N

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

  def __len__(self):
    return self.num

In [13]:
N = len(Data)-1
ferdousi = Ferdousi(torch.from_numpy(Data))
loader_train = DataLoader(ferdousi, batch_size= 128, sampler= sampler.SubsetRandomSampler(range(int(0.8*0.8*N))))
loader_valid = DataLoader(ferdousi, batch_size= 128, sampler= sampler.SubsetRandomSampler(range(int(0.8*0.8*N), int(0.8*N))))
loader_test = DataLoader(ferdousi, batch_size= 128, sampler= sampler.SubsetRandomSampler(range(int(0.8*N), N)))

In [14]:
def check_accuracy(loader, word_embed, generator, print_result = False, stage= 'validation'):
  num_samples = 0
  loss = 0
  generator.model.to(device)
  generator.model.eval()  # set model to evaluation mode
  with torch.no_grad():
    for x, y in loader:
      x ,y =  x.reshape((len(x),-1)).to(device= device), y.to(device)
      scores0 = generator.model.transformer(x).last_hidden_state
      yscores0 = word_embed(y)
      scores = generator.model.lm_head(scores0)
      loss0 = nn.functional.mse_loss(scores0, yscores0)
      loss1 = nn.functional.cross_entropy(scores.view(-1, scores.shape[-1]), y.view(-1,))
      loss_batch = loss1 + loss0
      # loss_batch = nn.functional.cross_entropy(scores.view(-1, scores.shape[-1]), y.view(-1,))
      _, preds = scores.max(1)
      num_samples += preds.size(0)
      loss += loss_batch * preds.size(0)
    loss = loss / num_samples
    if(print_result):
      print(f'for {stage} dataset: loss = {loss}')
  return loss.item()

In [15]:
def train_model(generator, word_embed, loader_train, valid_loader, epochs, lr = 1e-3):
  generator.model.to(device)
  word_embed.to(device)
  optimizer = torch.optim.Adam(generator.model.parameters(), lr = lr)
  num_batches = len(loader_train)
  val_result = []
  train_result = []
  models = []
  for epoch in range(epochs):
    generator.model.to(device)
    batches = tqdm_notebook(enumerate(loader_train), total= num_batches, desc= f'epoch {epoch} : ')
    for t, (x,y) in batches:
      generator.model.train()
      x ,y = x.reshape((len(x),-1)).to(device= device), y.to(device)
      scores0 = generator.model.transformer(x).last_hidden_state
      yscores0 = word_embed(y)
      scores = generator.model.lm_head(scores0)
      loss0 = nn.functional.mse_loss(scores0, yscores0)
      loss1 = nn.functional.cross_entropy(scores.view(-1, scores.shape[-1]), y.view(-1,))
      loss = loss1 + loss0
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    train_result.append(check_accuracy(loader_train, word_embed, generator, print_result= True, stage = 'train'))
    val_result.append(check_accuracy(valid_loader, word_embed, generator, print_result= True))
    # model_cpu = generator.model.to('cpu')
    # models.append(copy.deepcopy(generator))
    generator.model.save_pretrained("your_model_directory")
    torch.save(generator, f"/content/drive/MyDrive/Deep/HW4/Q3/gene{epoch}")
    print()
  return train_result, val_result, models

In [18]:
# generator = pipeline(task= "text-generation", model= "HooshvareLab/gpt2-fa")
generator1 = torch.load(path+'/gene19')
word_embed = copy.deepcopy(generator.model.transformer.wte)

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

In [59]:
train_result, val_result, models = train_model(generator1, word_embed, loader_train, loader_valid, epochs = 20, lr = 5e-4)

epoch 0 :   0%|          | 0/249 [00:00<?, ?it/s]

for train dataset: loss = 1.0453435182571411
for validation dataset: loss = 2.044628620147705



epoch 1 :   0%|          | 0/249 [00:00<?, ?it/s]

for train dataset: loss = 0.9192458987236023
for validation dataset: loss = 2.130769729614258



epoch 2 :   0%|          | 0/249 [00:00<?, ?it/s]

...

for train dataset: loss = 0.5084850192070007
for validation dataset: loss = 3.0415399074554443



epoch 18 :   0%|          | 0/249 [00:00<?, ?it/s]

for train dataset: loss = 0.4980129599571228
for validation dataset: loss = 3.0775551795959473



epoch 19 :   0%|          | 0/249 [00:00<?, ?it/s]

for train dataset: loss = 0.4934912323951721
for validation dataset: loss = 3.1197147369384766



In [68]:
generator1 = torch.load(path+'/gene19')

In [69]:
loss_train = check_accuracy(loader_test, word_embed, generator1, print_result= True, stage = 'test')

for test dataset: loss = 3.125551462173462


In [70]:
tocpu = generator1.model.to('cpu')

In [71]:
torch.save(generator1, f"/content/drive/MyDrive/Deep/HW4/Q3/gene16")

Test resault for sum samples:
first beit is input sample and second beit is the output of the generator

In [72]:
x, y = next(iter(loader_test))
scores = generator1.model(x[:5]).logits
outs = scores.argmax(2)
for i in range(5):
  input = tokenizer.decode(x[i])
  input = ' '.join(input.split('<|endoftext|>'))
  output = tokenizer.decode(outs[i])
  output = ' '.join(output.split('<|endoftext|>'))
  print(input)
  print(output)
  print()

هوا گفتی از نیزه چون بیشه گشت      خور از گرد اسپان پراندیشه گشت     
به دریای خون خون و و و را     بیر خروشان گردد از را       

همی گفت رادا سرا موبدا     ردا نامدارا یلا بخردا     
چومی بر دید روان  گنج      همه پر کار باد به کار       

سربدره های کهن بسته شد      وزان بند روشن دلم خسته شد       
بمیدارانداران شهر        علی دیلمی بر پیشاست بهر       

نگه کن بمن تا مرا نیز هست     اگر هست بیهوده منمای دست       
بگرش کاینداران        شوم شاد اگر رای فرخ نهی کجاست      

و گر باره خواهی روانم تراست      گروگان کنم جان بدان کت هواست      
بایم گشت خویش پیش ز بزرگیم     چو از که کن کینه ازوی      

